### Import thư viện

In [99]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import time
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

In [31]:
def get_feature_importance(table, name, model, X_test, y_test, time):
    table.append([name, mean_squared_error(y_test, model.predict(X_test))/ y_test.var(), r2_score(y_test, model.predict(X_test)), time])
    return table

In [58]:
def normalized(f):
    return (f - f.min(axis=0)) / (f.max(axis=0) - f.min(axis=0))

### Data preprocessing

In [32]:
recipes = pd.read_csv("epi_r.csv")
recipes = recipes.dropna()
recipes

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,The Best Blts,4.375,948.0,19.0,79.0,1042.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,Parmesan Puffs,3.125,28.0,2.0,2.0,64.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,Artichoke and Parmesan Risotto,4.375,671.0,22.0,28.0,583.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,Turkey Cream Puff Pie,4.375,563.0,31.0,38.0,652.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20050,Snapper on Angel Hair with Citrus Cream,4.375,631.0,45.0,24.0,517.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
X = recipes.drop([i for i in recipes.columns[:12]], axis=1)
y = recipes['calories']
print(X.shape, y.shape)

(15864, 668) (15864,)


In [88]:
y

0        426.0
1        403.0
2        165.0
4        547.0
5        948.0
         ...  
20047     28.0
20048    671.0
20049    563.0
20050    631.0
20051    560.0
Name: calories, Length: 15864, dtype: float64

Dùng PCA để giảm chiều dữ liệu

In [89]:
pca = PCA(n_components=34)   # Khởi tạo PCA với 34 thành phần chính
pca.fit(X)  # Fit PCA với dữ liệu
X_pca = pca.transform(X)    # Chuyển đổi dữ liệu sang thành phần chính đầu tiên
print(X_pca.shape, y.shape)

(15864, 34) (15864,)


In [63]:
pca = PCA(n_components=10)   # Khởi tạo PCA với 10 thành phần chính
pca.fit(X)  # Fit PCA với dữ liệu
X_pca = pca.transform(X)    # Chuyển đổi dữ liệu sang thành phần chính đầu tiên
print(X_pca.shape, y.shape)

(15864, 10) (15864,)


### Normalize

In [84]:
# Standardize the features
X = normalized(X)
X_pca = normalized(X_pca)
y = normalized(y)

In [76]:
y

0        1.414755e-05
1        1.338372e-05
2        5.479685e-06
4        1.816599e-05
5        3.148328e-05
             ...     
20047    9.298860e-07
20048    2.228405e-05
20049    1.869735e-05
20050    2.095565e-05
20051    1.859772e-05
Name: calories, Length: 15864, dtype: float64

### Split train/test

In [90]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(12691, 668) (12691,) (3173, 668) (3173,)


In [91]:
X_pca_train, X_pca_test, y_pca_train, y_pca_test = train_test_split(X_pca, y, test_size=0.2, random_state=23)
print(X_pca_train.shape, y_pca_train.shape, X_pca_test.shape, y_pca_test.shape)

(12691, 34) (12691,) (3173, 34) (3173,)


### Training 

#### Sau khi dùng PCA

In [92]:
# Logistic Regression with L1 regularization
print("Training l1...")
model_l1 = LogisticRegression(penalty='l1', solver='liblinear')
%time model_l1.fit(X_pca_train, y_pca_train)

Training l1...
CPU times: total: 5.02 s
Wall time: 32.7 s


LogisticRegression(penalty='l1', solver='liblinear')

In [93]:
# Logistic Regression with L2 regularization
print("Training l2...")
model_l2 = LogisticRegression(penalty='l2', solver='liblinear')
%time model_l2.fit(X_pca_train, y_pca_train)

Training l2...
CPU times: total: 12.1 s
Wall time: 1min 4s


LogisticRegression(solver='liblinear')

In [94]:
# Logistic Regression with Elastic Net regularization
print("Training l1 & l2...")
model_elasticnet = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5)
%time model_elasticnet.fit(X_pca_train, y_pca_train)

Training l1 & l2...
CPU times: total: 1min 50s
Wall time: 12min 15s


c:\Users\Mink\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga')

### Đánh giá

In [95]:
table = [["Optimier", "MSE", "R2 Score", "Time"]]

In [96]:
table = get_feature_importance(table, "Lasso_pca_34", model_l1, X_pca_test, y_pca_test, '57.9 s')
table = get_feature_importance(table, "Ridge_pca_34", model_l2, X_pca_test, y_pca_test, '1min 4s')
table = get_feature_importance(table, "ElasticNet_pca_34", model_elasticnet, X_pca_test, y_pca_test, '15min 36s')

In [55]:
pd.DataFrame(table)

,0,1,2,3
0,Optimier,MSE,R2 Score,Time
1,Lasso_pca_34,30386.995982,-30395.575741,57.9 s
2,Ridge_pca_34,2.361342,-1.362086,1min 4s
3,ElasticNet_pca_34,7595.836659,-7597.231311,15min 36s


In [102]:
# Define the parameter grid for each model
ridge_param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
lasso_param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
elasticnet_param_grid = {'C': [0.001, 0.01, 0.1, 1, 10], 'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]}

In [103]:
# Perform GridSearchCV for each model
ridge_grid = GridSearchCV(model_l1, ridge_param_grid, cv=5)
lasso_grid = GridSearchCV(model_l2, lasso_param_grid, cv=5)
elasticnet_grid = GridSearchCV(model_elasticnet, elasticnet_param_grid, cv=5)

In [104]:
# Fit the models
ridge_grid.fit(X_pca_train, y_pca_train)
lasso_grid.fit(X_pca_train, y_pca_train)
elasticnet_grid.fit(X_pca_train, y_pca_train)

c:\Users\Mink\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\Mink\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\Mink\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [ ]:
# Get the best hyperparameters and scores for each model
ridge_best_params = ridge_grid.best_params_
ridge_best_score = ridge_grid.best_score_
lasso_best_params = lasso_grid.best_params_
lasso_best_score = lasso_grid.best_score_
elasticnet_best_params = elasticnet_grid.best_params_
elasticnet_best_score = elasticnet_grid.best_score_

In [ ]:
# Print the results
print("Ridge Regression:")
print("Best Parameters:", ridge_best_params)
print("Best Score:", ridge_best_score)
print()
print("Lasso Regression:")
print("Best Parameters:", lasso_best_params)
print("Best Score:", lasso_best_score)
print()
print("ElasticNet Regression:")
print("Best Parameters:", elasticnet_best_params)
print("Best Score:", elasticnet_best_score)